# Global Liquidity & Productivity Dashboard (Free Data Edition)

Run this notebook end-to-end to fetch free macro/market data, compute liquidity & productivity indices, generate visualizations, and export CSV/PNG outputs. The workflow is designed for Google Colab or local Python 3.10+ environments.

In [ ]:
%pip install -r ../requirements.txt --quiet


In [ ]:
import sys
from pathlib import Path

import pandas as pd

PROJECT_ROOT = Path('..').resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.data_fetch import DataFetcher, FetchConfig
from src.compute_indices import (
    combine_components,
    compute_absorption_indices,
    compute_creation_index,
    compute_flow_index,
    compute_productivity_index,
    compute_rolling_pca,
    detect_regimes,
    scenario_tree,
)
from src.visualize import (
    FigurePaths,
    build_regional_heatmap,
    plot_absorption_efficiency,
    plot_composite,
    plot_creation_flow_lines,
    plot_four_quadrant_panel,
    traffic_light_dashboard,
)

config_path = PROJECT_ROOT / 'config' / 'config.json'
fetcher = DataFetcher(FetchConfig.load(config_path))
outputs = FigurePaths(fetcher.config.outputs_dir)


## 1. Fetch data

In [ ]:
creation_series = {
    "fed": fetcher.fetch_fed_balance_sheet(),
    "ecb": fetcher.fetch_ecb_total_assets(),
    "boj": fetcher.fetch_boj_total_assets(),
    "pboc": fetcher.fetch_pboc_m2(),
    "imf": fetcher.fetch_imf_broad_money(),
    "tga": fetcher.fetch_tga_balance(),
    "ust": fetcher.fetch_ust_issuance(),
}

flow_inputs = {
    "fx": fetcher.fetch_market_series([fetcher.config.section("flow")["fx_ticker"]], "fx").squeeze("columns"),
    "rates": fetcher.fetch_market_series([fetcher.config.section("flow")["rates_ticker"]], "rates").squeeze("columns"),
    "credit": fetcher.fetch_market_series([fetcher.config.section("flow")["credit_ticker"]], "credit").squeeze("columns"),
}

flow_equities = fetcher.fetch_market_series(fetcher.config.section("flow").get("equity_tickers", []), "equity")
flow_bonds = fetcher.fetch_market_series(fetcher.config.section("flow").get("bond_tickers", []), "bond")
flow_commodities = fetcher.fetch_market_series(fetcher.config.section("flow").get("commodity_tickers", []), "commodity")
stablecoins = fetcher.fetch_stablecoin_market_cap()

# Utilization & consumption placeholders (extend as data coverage improves)
world_bank_gfcf = fetcher.fetch_world_bank_indicator(fetcher.config.section("utilization").get("world_bank_gfcf_code", ""))
oecd_capex = fetcher.fetch_oecd_series(fetcher.config.section("utilization").get("oecd_capex_series", ""))
pmi_new_orders = fetcher.fetch_oecd_series(fetcher.config.section("utilization").get("pmi_series", {}).get("new_orders", ""))
pmi_inventories = fetcher.fetch_oecd_series(fetcher.config.section("utilization").get("pmi_series", {}).get("inventories", ""))
pmi_spread = (pmi_new_orders - pmi_inventories).dropna()

retail_sales = fetcher.fetch_retail_sales(fetcher.config.section("consumption").get("retail_sales", {}))

creation_series = {k: v.dropna() for k, v in creation_series.items() if isinstance(v, pd.Series) and not v.empty}


## 2. Compute indices

In [ ]:
creation_df = compute_creation_index({k: v for k, v in creation_series.items() if k in ["fed", "ecb", "boj", "pboc", "imf"]})
flow_inputs.update({
    "commodities": flow_commodities,
})
flow_df = compute_flow_index(flow_inputs, flow_equities, flow_bonds, stablecoins)

utilization_series = (world_bank_gfcf.combine_first(oecd_capex).combine_first(pmi_spread)).dropna()
consumption_series = retail_sales.mean(axis=1) if not retail_sales.empty else pd.Series(dtype=float)
absorption_df = compute_absorption_indices(utilization_series, consumption_series)

productivity_df = compute_productivity_index(
    tfp=creation_series.get("imf", pd.Series(dtype=float)).resample("A").last(),
    rd=world_bank_gfcf,
    ai_proxy=creation_series.get("ust", pd.Series(dtype=float)).pct_change().dropna(),
)

weights = fetcher.config.section("weights")
composite_df = combine_components(
    creation_df["creation_z"],
    flow_df["flow_index"],
    absorption_df["utilization_z"],
    absorption_df["consumption_z"],
    productivity_df["productivity_momentum"],
    weights,
)

creation_df.to_csv(outputs.path("creation_index.csv"))
flow_df.to_csv(outputs.path("flow_index.csv"))
absorption_df.to_csv(outputs.path("absorption_index.csv"))
productivity_df.to_csv(outputs.path("productivity_momentum.csv"))
composite_df.to_csv(outputs.path("composite_index.csv"))


## 3. Rolling PCA, regimes, and scenarios

In [ ]:
pca_input = composite_df[["creation", "flow", "utilization", "consumption", "efficiency"]]
pca_result = compute_rolling_pca(pca_input.dropna(), window=fetcher.config.pca_window)

rolling_pca_path = outputs.path("rolling_pca.csv")
pca_result.scores.to_csv(rolling_pca_path)

regimes = detect_regimes(composite_df["composite_index"], k_regimes=fetcher.config.regime_states)
if not regimes.empty:
    regimes.to_csv(outputs.path("regimes.csv"))

latest_levels = composite_df.iloc[-1][["creation", "flow", "utilization", "consumption", "efficiency"]]
scenario_df = scenario_tree(latest_levels.to_dict(), fetcher.config.scenario_shocks, weights)
scenario_df.to_csv(outputs.path("scenario_tree.csv"))
scenario_df


## 4. Visualizations

In [ ]:
plot_creation_flow_lines(creation_df, flow_df, outputs)
plot_absorption_efficiency(absorption_df, productivity_df, outputs)
plot_composite(composite_df, outputs)
plot_four_quadrant_panel(creation_df["creation_z"], absorption_df["absorption_index"], outputs)

regional_map = fetcher.config.section("visualization").get("regional_etf_map", {})
if regional_map:
    regional_prices = fetcher.fetch_market_series(regional_map.values(), "regional")
    regional_prices.columns = regional_map.keys()
    build_regional_heatmap(regional_prices, windows=[3, 6, 12], outputs=outputs)

delta_composite = composite_df["composite_index"].diff().iloc[-1] if len(composite_df) > 1 else 0
z_scores = latest_levels.to_dict()
traffic_light_dashboard(z_scores, delta_composite, outputs)


## 5. Summary

In [ ]:
summary = {
    "latest_date": str(composite_df.index[-1]),
    "composite": float(composite_df["composite_index"].iloc[-1]),
    "delta": float(delta_composite),
    "regime": "n/a" if regimes.empty else int(regimes["regime"].iloc[-1]),
}
summary
